# <center>Montar corpus dados de Pesquisa do Instituto Pasteur</center>

In [1]:
## Configurar opções de exibição do pandas para melhor visualizar os dados
from pathlib import Path
from getpass import getpass
from datetime import datetime
from IPython.display import clear_output
import pandas as pd, os, re, sys, time, json, subprocess
pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_rows', 600)

def find_repo_root(path='.', depth=10):
    ''' 
    Busca o arquivo .git e retorna string com a pasta raiz do repositório
    '''
    # Prevent infinite recursion by limiting depth
    if depth < 0:
        return None
    path = Path(path).absolute()
    if (path / '.git').is_dir():
        return path
    return find_repo_root(path.parent, depth-1)

delay = 10

## Definir a pasta de base do repositório local
base_repo_dir = find_repo_root()

## Sempre construir os caminhos usando os.path.join para compatibilidade WxL
folder_utils = os.path.join(base_repo_dir, 'utils')
folder_domain = os.path.join(base_repo_dir, 'source', 'domain')
folder_data_input = os.path.join(base_repo_dir, '_data', 'in_csv')
folder_data_output = os.path.join(base_repo_dir, '_data', 'out_json')

## Adicionar pastas locais ao sys.path para importar pacotes criados localmente
sys.path.append(folder_utils)
sys.path.append(folder_domain)
from lattes_scrapper import LattesScraper, SoupParser, DictToHDF5, attribute_to_be_non_empty
from scraper_fiocruz_ceara import FiocruzCearaScraper
from json_fle_manager import JSONFileManager as jfm
from scraper_pasteur import PasteurScraper

# from chromedriver_manager import ChromeDriverManager
# from environment_setup import EnvironmentSetup
# from lattes_html_parser import HTMLParser

print(f" Caminho base do repositório: {base_repo_dir}")
print(f"Arquivos de entrada de dados: {jfm.list_json_files(folder_data_input)}")

 Caminho base do repositório: c:\Users\marcos.aires\ppgcs
Arquivos de entrada de dados: ['dict_list_complete.json', 'dict_list_discents.json', 'dict_list_docents.json', 'dict_list_temp.json']


In [2]:
# Cria instância da classe ChromeDriverManager e verifica compatibilidade entre versões do Chrome e Chromedriver
# manager = ChromeDriverManager()
# manager.main()

In [3]:
base_url = "https://ceara.fiocruz.br/portal/"
scraper = FiocruzCearaScraper(base_url, find_repo_root())
fioce_data = scraper.scrape_areas()
fioce_data

[{'area_name': 'Saúde da Família',
  'area_desc': 'As pesquisas da área avaliam a Atenção Primária à Saúde. São estudos que investigam a atuação das equipes da Estratégia Saúde da Família em diferentes cenários e/ou processos de trabalho e/ou cuidado, em distintos ciclos de vida da população. As temáticas de pesquisa incluem o ensino e as práticas dos profissionais da saúde, saúde mental, condições crônicas, saúde bucal e saúde digital.',
  'area_plats': ['As diversas iniciativas que no momento caracterizam a atuação da Área de Saúde da Família na Fiocruz Ceará consolidam um processo de construção participativo, dinâmico e voltado a garantir respostas aos desafios de fortalecimento e qualificação dessa estratégia que, em virtude de sua magnitude, impõe à sociedade e governos, gigantesco esforços ao efetivo cumprimento de seu propósito. Entende-se pela sua magnitude: a cobertura de mais de 159 milhões de pessoas; mais de 1 milhão de postos de trabalho diretos; cobertura de quase todos o

In [4]:
base_url = "https://research.pasteur.fr"
scraper = PasteurScraper(base_url, find_repo_root())
json_data = scraper.generate_pasteur_report_html()

2024-05-10 08:30:26,541 - INFO - Obtendo os dados do site do Instituto Pasteur, aguarde...
2024-05-10 08:30:28,070 - INFO - 5 seções de dados principais extraídas
2024-05-10 08:30:28,073 - INFO - Obtendo áreas prioritárias de pesquisa, aguarde...
2024-05-10 08:30:38,609 - INFO - 5 áreas prioritárias em pesquisa extraídas
2024-05-10 08:30:38,613 - INFO - Obtendo os centros de referência, aguarde...
2024-05-10 08:30:41,461 - INFO - 5 centros extraídos
2024-05-10 08:30:41,464 - INFO - Obtendo os departamentos, aguarde...
2024-05-10 08:31:02,871 - INFO - 13 departamentos extraídos
2024-05-10 08:31:02,874 - INFO - Obtendo as plataformas, aguarde...
2024-05-10 08:31:58,681 - INFO - 35 plataformas extraídas
2024-05-10 08:31:58,698 - INFO - Relatório concluído!
2024-05-10 08:31:58,701 - INFO - Salvo em: c:\Users\marcos.aires\ppgcs\_data\out_json\report_pasteur_research.html


In [5]:
json_data.keys()

dict_keys(['priority_research', 'centers_data', 'heads_centers_data', 'departments_data', 'platforms_data'])

In [6]:
json_data.get('platforms_data')[0]

{'title': 'Collection of the Institut Pasteur (CIP)',
 'head_name': 'Dominique Clermont',
 'link': 'https://research.pasteur.fr/en/team/collection-of-the-institut-pasteur-cip/',
 'aditional_info': {'qte_keywords': '1',
  'qte_members': '8',
  'qte_projects': '4',
  'qte_publication': '105',
  'projects': [{'title': 'ISIDORe – Integrated Services for Infectious Diseases Outbreak Research',
    'head_name': 'Raquel Hurtado Ortiz (PI)',
    'status': 'Ongoing',
    'members': '4 Members',
    'description': 'Representation of MIRRI in ISIDORe The Biological Resource Center (CRBIP) represents the MIRRI infrastructure in ISIDORe. Our activities are focused on service provision (TA), but also in joint research activities. The ISIDORe project intends […]'},
   {'title': 'HoloZcan – Deep Learning Powered Holographic Microscopy for Biothreat Detection on Field',
    'head_name': 'Mariana Ferrari (PI)',
    'status': 'Ongoing',
    'members': '10 Members',
    'description': 'The project is a Re

In [7]:
for i in json_data.get('platforms_data'):
    platform_title = i.get('title')
    print('-'*50)
    print(platform_title.upper())
    try:
        project_list = i.get('aditional_info').get('projects')
        for j in project_list:
            title = j.get('title')
            head = j.get('head_name')
            status = j.get('status')
            description = j.get('description')
            print(f"  [{status}] {title} ({head})")
            print(f"            {description}")
    except:
        try:
            project_list = i.get('aditional_info').get('transversal_projects')
            for j in project_list:
                print(f"[Transversal Project] {j.get('title')}")
        except:
            print(f"            Projetos não encontrados para esta plataforma")

--------------------------------------------------
COLLECTION OF THE INSTITUT PASTEUR (CIP)
  [Ongoing] ISIDORe – Integrated Services for Infectious Diseases Outbreak Research (Raquel Hurtado Ortiz (PI))
            Representation of MIRRI in ISIDORe The Biological Resource Center (CRBIP) represents the MIRRI infrastructure in ISIDORe. Our activities are focused on service provision (TA), but also in joint research activities. The ISIDORe project intends […]
  [Ongoing] HoloZcan – Deep Learning Powered Holographic Microscopy for Biothreat Detection on Field (Mariana Ferrari (PI))
            The project is a Research and Innovation Action, its funding originates from the H2020-EU.3.7.5. – Increase Europe’s resilience to crises and disasters budget line, topic : SU-DRS04-2019-2020 – Chemical, biological, radiological and nuclear (CBRN) […]
  [Ongoing] IS_MIRRI21 – Implementation and Sustainability of Microbial Resource Research Infrastructure for the 21st century (Raquel Hurtado Ortiz (

In [8]:
for i in json_data.get('departments_data'):
    for j in i.get('aditional_info').get('teams'):
        print(j)
        print(f"  {j.get('head_name'):>35}: <b>{j.get('team_name')}</b>")
    print('-'*150)

{'team_name': 'Biological Image Analysis', 'head_name': 'Jean-Christophe Olivo-Marin', 'url': 'https://research.pasteur.fr/en/team/bioimage-analysis/'}
          Jean-Christophe Olivo-Marin: <b>Biological Image Analysis</b>
{'team_name': 'Biology of infection', 'head_name': 'Marc Lecuit', 'url': 'https://research.pasteur.fr/en/team/biology-of-infection/'}
                          Marc Lecuit: <b>Biology of infection</b>
{'team_name': 'Cell Polarity, Migration And Cancer', 'head_name': 'Sandrine Etienne-Manneville', 'url': 'https://research.pasteur.fr/en/team/cell-polarity-migration-and-cancer__trashed/'}
          Sandrine Etienne-Manneville: <b>Cell Polarity, Migration And Cancer</b>
{'team_name': 'Cellular Biology of Microbial Infection', 'head_name': 'Agathe Subtil', 'url': 'https://research.pasteur.fr/en/team/cellular-biology-of-microbial-infection/'}
                        Agathe Subtil: <b>Cellular Biology of Microbial Infection</b>
{'team_name': 'Chromatin and Infection', 'hea

# Testes iniciais

## Recuperar dados quantitativos gerais

In [ ]:
# scraper = PasteurScraper("https://research.pasteur.fr", base_repo_dir)
# main_data = scraper.scrape_main_page_quantitative_data()
# print(f"{len(main_data)} seções de dados principais extraídas")
# for i,j in main_data.items():
#     print(f"{i:>25}: {j}")

## Recuperar dados das áreas prioritárias de pesquisa

In [ ]:
# from pprint import pprint
# priority_research = scraper.scrape_priority_scientific_areas()
# filepath = os.path.join(folder_data_output,'priority_research.json')
# scraper.save_to_json(priority_research, filepath)

# print(f"{len(priority_research)} prioridades principais em pesquisa extraídas")
# for i in priority_research:
#     print(f"  {i.get('area_name'):>50}: {i.get('team_count')}")
# print()

In [ ]:
# priority_research

## Recuperar dados dos deparamentos do Pasteur

In [ ]:
# departments_data = scraper.scrape_department_data()
# filepath = os.path.join(folder_data_output,'departments_data.json')
# scraper.save_to_json(departments_data, filepath)

# print(f"{len(departments_data)} departamentos extraídos")
# for i in departments_data:
#     print(f"  {i.get('head_name'):>25}: {i.get('department_name')}")

In [ ]:
# departments_data

## Recuperar dados dos centros do Pasteur

In [ ]:
# heads_centers_data = scraper.scrape_centers()
# centers_data = scraper.scrape_centers_data()
# filepath = os.path.join(folder_data_output,'centers_data.json')
# scraper.save_to_json(centers_data, filepath)

# print(f"{len(centers_data)} centros extraídos")
# for i in heads_centers_data:
#     print(f"  {i.get('head_name'):>35}: {i.get('title')}")
# for i in centers_data:
#     print(f"\n{i.get('center_title')} ({len(i.get('teams')):02} times associados)")
#     for team in i.get('teams'):
#         print(f"  {team.get('head_name'):>35}: {team.get('title')}")

## Recuperar dados das plataformas do Pasteur

In [ ]:
# platforms_data = scraper.scrape_platforms()
# filepath = os.path.join(folder_data_output,'platforms_data.json')
# scraper.save_to_json(platforms_data, filepath)

# print(f"{len(platforms_data)} plataformas extraídas")
# for i in platforms_data:
#     print(f"  {i.get('head_name'):>25}: {i.get('title')}")

## Recuperar dados dos times do Pasteur

In [ ]:
# teams_data = scraper.scrape_teams_data()
# print(f"{len(teams_data)} times extraídos")
# for i in teams_data:
#     print(f"  {i.get('head_name'):>25}: {i.get('title')}")

In [ ]:
# filepath = os.path.join(folder_data_output,'teams_data.json')
# scraper.save_to_json(teams_data, filepath)

In [ ]:
# centers_data[1]

In [ ]:
# departments_data[-1]

In [ ]:
# platforms_data[-1]